# A Tour of bitformat

A number of classes are available in bitformat to store and manipulate binary data.

* ``Bits`` - An immutable container of binary data.
* ``Dtype`` - A data type that gives an interpretation to binary data.
* ``Array`` - A container for contiguously allocated `Bits` objects with the same `Dtype`.

These are the building blocks for more complex fields that can be used to make a binary format.

* ``Field`` - Either one value or an array, with a single data type, with an optional name and value.
* ``Format`` - A sequence of other FieldTypes, with an optional name.

For this tour we'll first install `bitformat` and import these classes::


In [ ]:
!pip install git+https://github.com/scott-griffiths/bitformat
from bitformat import Bits, Dtype, Array, Field, Format

Note that we are pip installing directly from the repository, as the library is still in alpha so PyPI probably won't have the latest version.

## Bits

The ``Bits`` class represents an immutable sequence of bits, similar to how the built-in ``bytes`` is an immutable sequence of bytes,
and a ``str`` is an immutable sequence of characters.

There are several builder class methods used to create ``Bits`` objects.

| Method name           | Description                                |
|-----------------------|--------------------------------------------|
| `Bits.pack(dtype, value)` | Combine a data type with a value.        |
| `Bits.from_string(s)` | Use a formatted string.                    |
| `Bits.from_bytes(b)`  | Directly from a `bytes` object.            |
| `Bits.from_iterable(i)` | Converts each element to a single bit.    |
| `Bits.zeros(n)`       | Initialise with zero bits.                 |
| `Bits.ones(n)`        | Initialise with one bits.                  |
| `Bits.join(iterable)` | Concatenate from an iterable such as a list.|


The ``Bits`` constructor can be used as a shortcut for the ``from_string`` method, so ``Bits(s)`` and ``Bits.from_string(s)`` are equivalent.

Creating from a string is often convenient and quite powerful.
The string can be a binary, octal or hexadecimal literal by starting with ``'0b'``, ``'0o'`` or ``'0x'`` respectively.
It can be a string that uses various data types of integer or floating point values, and it can be a sequence of tokens separated by commas. ::

In [ ]:
a = Bits('0b110')  # A 3-bit binary string
b = Bits('0xabcde')  # A 20-bit hexadecimal string
c = Bits('bytes=b"abcdefgh"') # An 8 byte bytes object
d = Bits('f32=13.5')  # A 32-bit IEEE floating point number
e = Bits('i7=-31')  # A 7-bit signed integer
f = Bits('0b001, u32=90, 0x5e')  # Three Bits objects concatenated together


Finally a data type can be used to create a `Bits` object by using the `pack` class method. ::

In [ ]:
g = Bits.pack('u8', 65)  # An 8-bit unsigned integer with the value 65
h = Bits.pack('hex', 'abcde')  # A 20-bit hexadecimal string
i = Bits.pack('bytes', b'hello')  # A 40-bit binary string
j = Bits.pack('f16', -13.81)  # A 16-bit IEEE floating point number



The first parameter of ``pack`` is the data-type, which can be either a ``Dtype`` or a string that can be used to create one.
The second parameter is a value that makes sense for that data type, which could be a binary string, a floating point number, an integer etc. depending on the ``Dtype``.

Once you've created your ``Bits`` object there is a rich API for manipulating and interpreting the data.
One fundamental thing to do is to interpret the binary data according to a format or data-type; essentially the opposite to how the ``pack`` method works. ::

In [ ]:
g.unpack('u8')  # Returns 65

In [ ]:
h.unpack(['hex20'])  # Returns ['abcde']


The ``unpack`` method is quite powerful and is a bit of a sledgehammer for these simple cases, so as a shortcut you can use properties that are available for simple dtypes. ::

In [ ]:
g.u

In [ ]:
h.hex  # Returns 'abcde'


Of course the ``Bits`` object is just a collection of bits and doesn't know how it was created, so any interpretation that makes sense is allowed ::

In [ ]:
a.unpack('oct')  # an octal string

In [ ]:
b.unpack('u')  # an unsigned int

In [ ]:
c.unpack('f_le64')  # a 64-bit little-endian IEEE floating point number

In [ ]:
d.unpack('hex')

In [ ]:
e.unpack('bin')


In places where a ``Bits`` is expected, a formatted string that can be used to more conveniently create the `Bits` object.
For example, if ``a`` is a ``Bits`` object, instead of ::

    a += Bits.pack('u8', 65)

you can equivalently write ::

    a += 'u8 = 65'

Some examples of strings that can be converted to `Bits` objects:

* ``'0b00110'``: A binary string.
* ``'0x3fff0001'``: A hexadecimal string.
* ``'i15=-401'``: A 15 bit signed integer representing the number -401.
* ``'f64=1.3e5'``: A 64 bit floating point number representing 130000.
* ``'0b001, u32=90, 0x5e'``: A sequence of bits that represent a 3-bit binary number, a 32-bit unsigned integer and a 8-bit hexadecimal number.

## Dtype

The dtype (or data type) gives an interpretation to binary data.
Most of these have a type and a bit-length, and are usually created when needed from a string as in the previous section.

For example the data-type representing an unsigned integer of length 4 bits can be created by either ``Dtype('u', 4)`` or by using the string ``'u4'`` when a `Dtype` is required as a parameter.

Some examples of the data-types names available are:

| Dtype string | Description |
|--------------|-------------|
| `'u10'`      | A 10-bit unsigned integer |
| `'i7'`       | A 7-bit signed two's complement integer |
| `'f32'`      | A 32-bit IEEE floating point number |
| `'bin4'`     | A 4-bit binary string |
| `'hex12'`    | A 12-bit hexadecimal string (i.e. 3 hex characters) |
| `'bool'`     | A single bit boolean |
| `'bits5'`    | A Bits instance of length 5 bits |
| `'bytes20'`  | 20 bytes of data |
| `'pad8'`     | Pad bits that have no interpretation |

Note that there are no unnatural restrictions on the length of a dtype.
If you want a 3-bit integer or 1001 padding bits then that's as easy to do as any other length.


## Array

The `Array` class is used as a container for contiguously allocated `Bits` objects with the same `Dtype`.

`Array` instances act very like an ordinary Python array, but with each element being a fixed-length dtype.


